# Create a custom local circuit(Cirq Circuit) and run the custom circuit on a custom simulator(Cirq Simulator)

1. Install the required dependencies
2. Define CustomProgramContex inheriting the AbstractProgramContext
3. Map custom circuit gates to the openqasm interpreted string
4. Pass the CustomProgramContex to an interpreter and build the circuit
5. Run the custom circuit on custom simulator/device

## Setup for desired circuit

In this example, the `cirq` library is installed as a prerequisite.

In [1]:
!pip install cirq

In [2]:
from typing import List, Tuple, Optional

import cirq
import numpy as np
from braket.ir.jaqcd.program_v1 import Results
from cirq import Circuit as CirqCircuit

from braket.default_simulator.openqasm.program_context import AbstractProgramContext
from braket.default_simulator.openqasm.interpreter import Interpreter

## Gate Mapping
Map OpenQASM strings to Cirq gates

In [3]:
from typing import Tuple

import cirq
from cirq import (
    bit_flip,
    PhaseFlipChannel,
    DepolarizingChannel,
    AmplitudeDampingChannel,
    GeneralizedAmplitudeDampingChannel,
    PhaseDampingChannel,
)

# cirq.XX, cirq.YY, and cirq.ZZ gates are not the same as Braket gates
CIRQ_GATES = {
    "i": cirq.I,
    "h": cirq.H,
    "x": cirq.X,
    "y": cirq.Y,
    "z": cirq.Z,
    "cnot": cirq.CNOT,
    "cz": cirq.CZ,
    "s": cirq.S,
    "t": cirq.T,
    "cphaseshift": cirq.cphase,
    "rx": cirq.rx,
    "ry": cirq.ry,
    "rz": cirq.rz,
    "swap": cirq.SWAP,
    "iswap": cirq.ISWAP,
    "ccnot": cirq.CCNOT,
    "cswap": cirq.CSWAP,
    "ccz": cirq.CCZ,
    "ccx": cirq.CCX,
    "measure": cirq.MeasurementGate,
}

CIRQ_NOISE_GATES = {
    "bit_flip": bit_flip,
    "phase_flip": PhaseFlipChannel,
    "depolarizing": DepolarizingChannel,
    "amplitude_damping": AmplitudeDampingChannel,
    "generalized_amplitude_damping": GeneralizedAmplitudeDampingChannel,
    "phase_damping": PhaseDampingChannel,
}


def get_cirq_qubits(qubits: Tuple[int]):
    return [cirq.LineQubit(int(qubit)) for qubit in qubits]

## Define ProgramContext

Inherit AbstractProgramContext and define the abstract methods

- The `AbstractProgramContext` class represents the context or state of an interpreter for a program. It provides a framework for managing symbols, variables, gates, subroutines, qubit mappings, and the circuit.


- The AbstractProgramContext class is an abstract base class (ABC) that defines the common structure and functionality expected in a program context. 

- It serves as a blueprint for creating concrete program contexts by defining a set of abstract methods and attributes that must be implemented by the subclasses. 

- Key attributes of the AbstractProgramContext include the symbol table, variable table, gate table, subroutine table, qubit mapping table, and the program circuit. 

- The circuit attribute represents the intermediate representation (IR) of the program that is built and handed off to the simulator or executor. 

- Various other methods, such as add_gate, get_gate_definition, is_user_defined_gate, add_subroutine, and get_subroutine_definition, enable gate and subroutine management within the program. 

- The AbstractProgramContext defines abstract methods for adding results, phase instructions, gate instructions, custom unitaries, and noise instructions to the program. 

In [4]:
class CirqProgramContext(AbstractProgramContext):
    def __init__(self, circuit: Optional[CirqCircuit] = None):
        """
        Args:
            circuit (Optional[Circuit]): A partially-built circuit to continue building with this
                context. Default: None.
        """
        super().__init__()
        self._circuit = circuit or CirqCircuit()

    @property
    def circuit(self) -> CirqCircuit:
        return self._circuit

    def is_builtin_gate(self, name: str) -> bool:
        """Whether the gate is currently in scope as a built-in Braket gate.

        Args:
            name (str): name of the built-in Braket gate

        Returns:
            bool: return TRUE if it is a built-in gate else FALSE.
        """
        user_defined_gate = self.is_user_defined_gate(name)
        return name in CIRQ_GATES and not user_defined_gate

    def add_gate_instruction(
            self, gate_name: str, target: Tuple[int], params, ctrl_modifiers: List[int], power: int
    ):
        """Add Braket gate to the circuit.

        Args:
            gate_name (str): name of the gate.
            target (Tuple[int]): control_qubits + target_qubits.
            ctrl_modifiers (List[int]): Quantum state on which to control the
                operation. Must be a binary sequence of same length as number of qubits in
                `control_qubits` in target. For example "0101", [0, 1, 0, 1], 5 all represent
                controlling on qubits 0 and 2 being in the \\|0⟩ state and qubits 1 and 3 being
                in the \\|1⟩ state.
            power(float): Integer or fractional power to raise the gate to.
        """
        qubits = get_cirq_qubits(target)
        target_cirq_qubits = qubits[len(ctrl_modifiers):]
        control_qubits = qubits[: len(ctrl_modifiers)]

        if params:
            gate = CIRQ_GATES[gate_name](*params).on(*target_cirq_qubits)
        else:
            gate = CIRQ_GATES[gate_name].on(*target_cirq_qubits)
        gate = gate.controlled_by(*control_qubits, control_values=ctrl_modifiers)
        gate = gate ** power

        self._circuit.append(gate)

    def add_custom_unitary(
            self,
            unitary: np.ndarray,
            target: Tuple[int],
    ) -> None:
        """Add a custom Unitary instruction to the circuit

        Args:
            unitary (np.ndarray): unitary matrix
            target (Tuple[int]): control_qubits + target_qubits
        """
        qubits = get_cirq_qubits(target)
        instruction = cirq.MatrixGate(unitary).on(*qubits)
        self._circuit.append(instruction)

    def add_noise_instruction(self, noise_instruction: str, target: List[int], probabilities: List[float]):
        """Method to add a noise instruction to the circuit

        Args:
            noise_instruction (str): The name of the noise operation
            target (List[int]): The target qubit or qubits to which the noise operation is applied.
            probabilities (List[float]): The probabilities associated with each possible outcome
                of the noise operation.
        """
        qubits = get_cirq_qubits(target)
        instruction = CIRQ_NOISE_GATES[noise_instruction](*probabilities).on(*qubits)
        self._circuit.append(instruction)

    def add_result(self, result: Results) -> None:
        """Add a result type to the circuit"""
        raise NotImplementedError
        
    def add_phase_instruction(self, target: Tuple[int], phase_value: int):
        raise NotImplementedError

## Basic gates example
This section showcases an example of basic gates. An OpenQASM program is defined, and a Circ circuit is built using the `Interpreter` and `CirqProgramContext` classes.

In [5]:
openqasm = """
            OPENQASM 3.0;
            bit[1] b;
            qubit[1] q;
            h q[0];
           """

basic_cirq_circuit = Interpreter(CirqProgramContext()).build_circuit(
        source=openqasm,
    )

basic_cirq_circuit.append(cirq.measure(*basic_cirq_circuit.all_qubits()))
print(basic_cirq_circuit)


0: ───H───M───


/var/folders/tb/2yh3yjmn3196046lsp7n2m5r0000gr/T/ipykernel_58596/2448392670.py:46: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if params:


## Noise Gate example

In [6]:
openqasm = """
            OPENQASM 3.0;
            bit[1] b;
            qubit[1] q;
            x q[0];
            #pragma braket noise bit_flip(0.1) q[0]
            b[0] = measure q[0];
            """
noise_cirq_circuit = Interpreter(CirqProgramContext()).build_circuit(
        source=openqasm,
    )
noise_cirq_circuit.append(cirq.measure(*basic_cirq_circuit.all_qubits()))
print(noise_cirq_circuit)

0: ───X───BF(0.1)───M───


/var/folders/tb/2yh3yjmn3196046lsp7n2m5r0000gr/T/ipykernel_58596/2448392670.py:46: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if params:


## Cirq Simulator Integration with Braket

The code snippet demonstrates the integration of Cirq's quantum circuit simulator with the Braket library to facilitate quantum circuit execution and simulation. The `CirqSimulator` class extends the `OpenQASMSimulator` class provided by Braket and defines methods to initialize and run Cirq simulations within the Braket environment.

#### `CirqSimulator` Class Overview

The `CirqSimulator` class is designed to enable seamless execution of quantum circuits using Cirq's simulator capabilities while harnessing the infrastructure and features of the Braket platform. The class inherits from `OpenQASMSimulator`, which acts as the foundation for simulator integration.

#### Key Methods

- `initialize_simulation()`: Initializes a Cirq simulator instance, preparing it for quantum circuit simulation.

- `run(circuit_ir, shots, *args, **kwargs)`: Accepts an OpenQASM program (`circuit_ir`), the number of shots (`shots`), and additional arguments/keywords for circuit execution. The method constructs a quantum circuit using Cirq, appends measurement operations, simulates the circuit, and returns the results in a Braket-compatible format.

- `execution_manager(*args, **kwargs)`: Creates and returns an instance of `LocalExecutionManager` specifically configured for the `CirqSimulator`. This allows managing the execution of circuits in a local simulation environment.

- `_get_measured_qubits(result)`: Internal method that extracts the qubits that were measured from the execution result.

- `_create_results_obj(circuit_ir, result)`: Internal method that constructs a Braket-compatible `GateModelTaskResult` object from the execution results.

- `properties`: Defines device properties, such as supported gates, modifiers, and pragmas. It also specifies execution windows and shots ranges for the simulator.

#### Device Capabilities

The `properties` property defines the capabilities of the `CirqSimulator`. It lists supported operations, modifiers, pragmas, and other device-related details that Braket can use to interface with the simulator effectively.

By integrating Cirq's powerful simulator capabilities with the Braket platform, the `CirqSimulator` class offers a versatile and user-friendly environment for simulating quantum circuits, leveraging the strengths of both libraries.


In [7]:
from typing import Union

from braket.task_result import (
    AnalogHamiltonianSimulationTaskResult,
    AnnealingTaskResult,
    GateModelTaskResult,
)

from braket.simulator import BraketSimulator
from braket.simulator.execution_manager import ExecutionManager

class LocalExecutionManager(ExecutionManager):
    """Manages the execution of a quantum program using a local simulator."""

    def __init__(self, simulator: BraketSimulator, *args, **kwargs):
        """Initialize the LocalExecutionManager.

        Args:
            simulator (BraketSimulator): The local simulator to use for quantum program execution.
            args: Additional positional arguments for configuring the simulation.
            kwargs: Additional keyword arguments for configuring the simulation.
        """
        self.simulator = simulator
        self.args = list(args)
        self.kwargs = kwargs

    def result(
        self,
    ) -> Union[GateModelTaskResult, AnnealingTaskResult, AnalogHamiltonianSimulationTaskResult,]:
        """Get the quantum task result.

        Returns:
            Union[GateModelQuantumTaskResult, AnnealingQuantumTaskResult,
            PhotonicModelQuantumTaskResult]:
            The result of the quantum task execution using the local simulator.
        """
        return self.simulator.run(*self.args, **self.kwargs)

    def cancel(self) -> None:
        """Cancel the quantum task.

        Raises:
            NotImplementedError: LocalExecutionManager does not support cancelling.
        """
        raise NotImplementedError("LocalQuantumTask does not support cancelling")

    def id(self) -> None:
        """Id of the quantum task.

        Raises:
            NotImplementedError: Id is not generated for LocalQuantumTask.
        """
        raise NotImplementedError("Id is not generated for LocalQuantumTask")


In [8]:
import sys
import uuid

import cirq
from braket.device_schema.simulators import (
    GateModelSimulatorDeviceCapabilities,
    GateModelSimulatorDeviceParameters,
)
from braket.ir.openqasm import Program as OpenQASMProgram
from braket.task_result import GateModelTaskResult, TaskMetadata, AdditionalMetadata
from cirq import Simulator


from braket.default_simulator.openqasm.interpreter import Interpreter
from braket.default_simulator.simulator import OpenQASMSimulator


class CirqSimulator(OpenQASMSimulator):
    DEVICE_ID = "cirq"

    def initialize_simulation(self):
        """
        Initialize cirq simulation.


        Returns:
            cirq.Simulator: Initialized simulation.
        """
        return Simulator()

    def run(self, circuit_ir: OpenQASMProgram, shots, *args, **kwargs):
        circuit = self.parse_program(circuit_ir).circuit
        circuit.append(cirq.measure(*circuit.all_qubits()))
        simulation = self.initialize_simulation()
        result = simulation.run(circuit, repetitions=shots)
        return self._create_results_obj(circuit_ir, result)
    
    def execution_manager(self, *args, **kwargs) -> LocalExecutionManager:
        return LocalExecutionManager(self, *args, **kwargs)
    
    def create_program_context(self) -> CirqProgramContext:
        return CirqProgramContext()

    def _get_measured_qubits(self, result):
        measured_qubits = set()
        for key in result.measurements.keys():
            measured_qubits.update([qubit[qubit.index("(") + 1 : -1] for qubit in key.split(",")])
        return list(measured_qubits)

    def _create_results_obj(
        self,
        circuit_ir: OpenQASMProgram,
        result: cirq.Result,
    ) -> GateModelTaskResult:
        return GateModelTaskResult.construct(
            taskMetadata=TaskMetadata(
                id=str(uuid.uuid4()),
                shots=result.repetitions,
                deviceId=self.DEVICE_ID,
            ),
            additionalMetadata=AdditionalMetadata(
                action=circuit_ir,
            ),
            measurements=list(list(result.measurements.values())[0]),
            measuredQubits=self._get_measured_qubits(result),
        )

    @property
    def properties(self):
        max_shots = sys.maxsize
        return GateModelSimulatorDeviceCapabilities.parse_obj(
            {
                "service": {
                    "executionWindows": [
                        {
                            "executionDay": "Everyday",
                            "windowStartHour": "00:00",
                            "windowEndHour": "23:59:59",
                        }
                    ],
                    "shotsRange": [0, max_shots],
                },
                "action": {
                    "braket.ir.openqasm.program": {
                        "actionType": "braket.ir.openqasm.program",
                        "version": ["1"],
                        "supportedOperations": [
                            # builtin Braket gates
                            "ccnot",
                            "cnot",
                            "cphaseshift",
                            "cswap",
                            "h",
                            "i",
                            "iswap",
                            "rx",
                            "ry",
                            "rz",
                            "s",
                            "swap",
                            "t",
                            "x",
                            "y",
                            "z",
                        ],
                        "supportedModifiers": [
                            {
                                "name": "ctrl",
                            },
                            {
                                "name": "negctrl",
                            },
                            {
                                "name": "pow",
                                "exponent_types": ["int", "float"],
                            },
                            {
                                "name": "inv",
                            },
                        ],
                        "supportedPragmas": [
                            "braket_unitary_matrix",
                            "braket_result_type_state_vector",
                            "braket_result_type_density_matrix",
                            "braket_result_type_sample",
                            "braket_result_type_expectation",
                            "braket_result_type_variance",
                            "braket_result_type_probability",
                            "braket_result_type_amplitude",
                            "braket_noise_bit_flip",
                            "braket_noise_depolarizing",
                            "braket_noise_generalized_amplitude_damping",
                            "braket_noise_phase_flip",
                            "braket_noise_phase_damping",
                        ],
                        "forbiddenPragmas": [
                            "braket_noise_amplitude_damping",
                            "braket_noise_kraus",
                            "braket_noise_pauli_channel",
                            "braket_noise_two_qubit_dephasing",
                            "braket_noise_two_qubit_depolarizing",
                            "braket_result_type_adjoint_gradient",
                        ],
                        "supportPhysicalQubits": False,
                        "supportsPartialVerbatimBox": False,
                        "requiresContiguousQubitIndices": True,
                        "requiresAllQubitsMeasurement": True,
                        "supportsUnassignedMeasurements": True,
                        "disabledQubitRewiringSupported": False,
                    },
                },
                "paradigm": {"qubitCount": 20},
                "deviceParameters": GateModelSimulatorDeviceParameters.schema(),
            }
        )

### Running a braket circuit on the local simulator defined above

In [9]:
from braket.circuits import Circuit
from braket.circuits.serialization import IRType
from braket.devices import LocalSimulator

dev = LocalSimulator(CirqSimulator())
braket_circuit = Circuit().h(0).cnot(0,1)
result = dev.run(braket_circuit, shots=10).result()
print("Result", result.measurements)

Result [[1 1]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]]


/var/folders/tb/2yh3yjmn3196046lsp7n2m5r0000gr/T/ipykernel_58596/2448392670.py:46: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if params:


## Add Cirq Ionq as device using the program context and task execution manager

In [10]:
!pip install cirq_ionq

### Cirq IonQ Execution Manager

The `CirqIonqExecutionManager` class serves as an execution manager, facilitating the execution of Cirq programs on IonQ devices using the Braket library. The class offers a comprehensive suite of functionalities for handling circuit creation, execution, result retrieval, and task management.

Execution Manager play the key role of maintaining all the task related queries.

#### Constructor

The constructor of the `CirqIonqExecutionManager` class initializes the manager with the following parameters:

- `device`: An instance of the IonQ quantum device on which to run the program.
- `program`: The OpenQASM program to be executed on the device.
- `shots`: The number of times the circuit should be executed.
- `*args` and `**kwargs`: Additional positional and keyword arguments to pass to the device creation function.

#### `result()` 

The `result()` method retrieves the results of the quantum execution. It returns a `GateModelTaskResult` object that includes metadata about the execution and the recorded measurements.

#### `cancel()` 

The `cancel()` method allows cancellation of an ongoing quantum execution on the IonQ device.

#### `state()` 

The `state()` method returns the current state of the quantum task. The states can include 'completed', 'canceled', 'failed', 'deleted', 'ready', 'submitted', and 'running'.

#### Internal Methods

The class includes two internal methods:

- `_get_measured_qubits(result)`: This method extracts the qubits that were measured from the execution result.
- `_create_results_obj(circuit_ir, result)`: An internal method that constructs a task result object from the execution results. The result includes metadata and measurements.

The `CirqIonqExecutionManager` class is designed to streamline the process of running Cirq programs on IonQ devices via the Braket library. It encapsulates key functionalities and provides an organized approach to managing quantum executions and analyzing outcomes.


In [11]:
from typing import Union
from braket.simulator.execution import ExecutionManager
import cirq_ionq

class CirqIonqExecutionManager(ExecutionManager):
    """Execution manager for running Cirq programs on IonQ devices.
    
    This class provides functionality to manage the execution of Cirq programs on IonQ quantum devices.
    It handles circuit creation, execution, and result retrieval, as well as providing methods to
    interact with the executed task.
    
    Args:
        device (IonQDevice): An instance of the IonQ quantum device on which to run the program.
        program (str): The OpenQASM program to be executed on the device.
        shots (int): The number of times the circuit should be executed.
        *args: Additional positional arguments passed to the device creation function.
        **kwargs: Additional keyword arguments passed to the device creation function.
    """

    def __init__(self, device,job, program,*args, **kwargs):
        """Initialize the execution manager with the provided device, program, and shot count."""
        self.job = job
        self.program = program
        self.device = device
    
#     def id(self):
#         return self.job

    def result(
            self,
    ) -> GateModelTaskResult:
        """Retrieve the results of the quantum execution.
    
        Returns:
            GateModelTaskResult: The results of the quantum execution, including metadata and measurements.
        """
        result = self.job.results().to_cirq_result()
        return self._create_results_obj(self.program, result)

    def cancel(self) -> None:
        """Cancel the quantum task.
        
        Cancels the ongoing quantum execution on the IonQ device.
        """
        self.job.cancel()

    def state(self) -> str:
        """Get the current state of the quantum task.
        
        Returns:
            str: The current state of the quantum task, e.g. 'completed', 'canceled', 'failed', 'deleted', 'ready', 'submitted', 'running'.
        """
        status_map = {
            "ready": "CREATED",
            "submitted": "QUEUED",
            "running": "RUNNING",
            "completed": "COMPLETED",
            "failed": "FAILED",
            "canceled": "CANCELLED",
        }
        status = self.job.status()
        return status_map.get(status, status)

    
    def _get_measured_qubits(self, result):
        measured_qubits = set()
        for key in result.measurements.keys():
            measured_qubits.update([qubit[qubit.index("(") + 1 : -1] for qubit in key.split(",")])
        return list(measured_qubits)

    def _create_results_obj(
        self,
        circuit_ir: OpenQASMProgram,
        result: cirq.Result,
    ) -> GateModelTaskResult:
        """Create a task result object from the execution results.
        
        Args:
            circuit_ir (OpenQASMProgram): The original OpenQASM program executed.
            result (cirq.Result): The execution result containing measurements.
            
        Returns:
            GateModelTaskResult: A constructed task result object containing metadata and measurements.
        """
        return GateModelTaskResult.construct(
            taskMetadata=TaskMetadata(
                id=str(uuid.uuid4()),
                shots=result.repetitions,
                deviceId=self.device.DEVICE_ID,
            ),
            additionalMetadata=AdditionalMetadata(
                action=circuit_ir,
            ),
            measurements=list(list(result.measurements.values())[0]),
            measuredQubits=self._get_measured_qubits(result),
        )

ModuleNotFoundError: No module named 'braket.simulator.execution'

In [ ]:
import sys
import uuid
import os

import cirq
from braket.device_schema.simulators import (
    GateModelSimulatorDeviceCapabilities,
    GateModelSimulatorDeviceParameters,
)
from braket.ir.openqasm import Program as OpenQASMProgram
from braket.task_result import GateModelTaskResult, TaskMetadata, AdditionalMetadata
from cirq import Simulator


from braket.default_simulator.openqasm.interpreter import Interpreter
from braket.default_simulator.simulator import OpenQASMSimulator
from braket.default_simulator.local_execution_manager import LocalExecutionManager


class CirqIonqDevice(OpenQASMSimulator):
    DEVICE_ID = "CirqIonqDevice"
    service = cirq_ionq.Service(api_key=os.environ.get('CIRQ_IONQ_APIKEY'))
    def create_job(self, circuit, *args, **kwargs):
        return self.service.create_job(circuit, *args, **kwargs)

    def run(self):
        # Since it runs on cloud
        pass

    def execution_manager(self, program, *args, **kwargs) -> CirqIonqExecutionManager:
        circuit = self.parse_program(program).circuit
        circuit.append(cirq.measure(*circuit.all_qubits()))
        job = self.service.create_job(circuit, *args, **kwargs)
        return CirqIonqExecutionManager(self,job, program, *args, **kwargs)

    def create_program_context(self) -> CirqProgramContext:
        return CirqProgramContext()

    @property
    def properties(self):
        max_shots = sys.maxsize
        return GateModelSimulatorDeviceCapabilities.parse_obj(
            {
                "service": {
                    "executionWindows": [
                        {
                            "executionDay": "Everyday",
                            "windowStartHour": "00:00",
                            "windowEndHour": "23:59:59",
                        }
                    ],
                    "shotsRange": [0, max_shots],
                },
                "action": {
                    "braket.ir.openqasm.program": {
                        "actionType": "braket.ir.openqasm.program",
                        "version": ["1"],
                        "supportedOperations": [
                            # builtin Braket gates
                            "ccnot",
                            "cnot",
                            "cphaseshift",
                            "cswap",
                            "h",
                            "i",
                            "iswap",
                            "rx",
                            "ry",
                            "rz",
                            "s",
                            "swap",
                            "t",
                            "x",
                            "y",
                            "z",
                        ],
                        "supportedModifiers": [
                            {
                                "name": "ctrl",
                            },
                            {
                                "name": "negctrl",
                            },
                            {
                                "name": "pow",
                                "exponent_types": ["int", "float"],
                            },
                            {
                                "name": "inv",
                            },
                        ],
                        "supportedPragmas": [
                            "braket_unitary_matrix",
                            "braket_result_type_state_vector",
                            "braket_result_type_density_matrix",
                            "braket_result_type_sample",
                            "braket_result_type_expectation",
                            "braket_result_type_variance",
                            "braket_result_type_probability",
                            "braket_result_type_amplitude",
                            "braket_noise_bit_flip",
                            "braket_noise_depolarizing",
                            "braket_noise_generalized_amplitude_damping",
                            "braket_noise_phase_flip",
                            "braket_noise_phase_damping",
                        ],
                        "forbiddenPragmas": [
                            "braket_noise_amplitude_damping",
                            "braket_noise_kraus",
                            "braket_noise_pauli_channel",
                            "braket_noise_two_qubit_dephasing",
                            "braket_noise_two_qubit_depolarizing",
                            "braket_result_type_adjoint_gradient",
                        ],
                        "supportPhysicalQubits": False,
                        "supportsPartialVerbatimBox": False,
                        "requiresContiguousQubitIndices": True,
                        "requiresAllQubitsMeasurement": True,
                        "supportsUnassignedMeasurements": True,
                        "disabledQubitRewiringSupported": False,
                    },
                },
                "paradigm": {"qubitCount": 20},
                "deviceParameters": GateModelSimulatorDeviceParameters.schema(),
            }
        )


### Quantum Circuit Execution Using Braket and Cirq IonQ

This code demonstrates how to create and execute a quantum circuit using the Braket library and the integration with Cirq IonQ. Here's a step-by-step breakdown of the code:

1. **Importing Dependencies**: The required classes and functions are imported from the Braket library. Specifically, `Circuit` is imported from `braket.circuits` for circuit construction, `Program` is imported from `braket.ir.openqasm` for defining quantum programs, and `BraketSchemaHeader` is imported from `braket.schema_common` for schema information.

2. **Creating a Cirq IonQ Device**: An instance of the `LocalSimulator` is created with the `CirqIonqDevice` as the target. This configuration sets up a local simulation environment using the Cirq IonQ device.

3. **Defining the Quantum Circuit**: A quantum circuit is constructed using the `Circuit` class from Braket. In this example, the circuit consists of a Hadamard gate applied to qubit 0 (`circuit.h(0)`) and a controlled-NOT (CNOT) gate between qubits 0 and 1 (`circuit.cnot(0, 1)`).

4. **Running the Quantum Circuit**: The constructed quantum circuit is executed on the Cirq IonQ device using the `run` method of the `cirq_ionq_device`. The `shots` parameter specifies the number of times the circuit is executed (20 shots in this case), simulating multiple runs of the circuit. The `target` parameter is set to "simulator," indicating that the simulation will be carried out on a local simulator.

5. **Result Handling**: The execution of the circuit results in a `task` object, which represents the execution task. You can retrieve and analyze the execution results using this task object.

This code provides a basic example of how to define and execute a quantum circuit using the Braket library in conjunction with Cirq IonQ's integration.


In [ ]:
from braket.ir.openqasm import Program
from braket.schema_common import BraketSchemaHeader


cirq_ionq_device = LocalSimulator(CirqIonqDevice())
braket_circuit = Circuit().h(0).cnot(0,1)
task = cirq_ionq_device.run(braket_circuit,shots =20, target ="qpu")

In [ ]:
print(task.state())

In [ ]:
print(task.result().measurements)